In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures # for polynomial features
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv(r"..\Cases\Glass Identification\Glass.csv")
le = LabelEncoder()
y = le.fit_transform(df['Type'])
X = df.drop('Type', axis=1)

In [9]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor

kfold = StratifiedKFold(shuffle=True, random_state=24, n_splits=5)

std_scaler = StandardScaler()
mm_scaler = MinMaxScaler()
knn = KNeighborsClassifier()

pipe = Pipeline([('SCL', None), ('KNN', knn)])

print(pipe.get_params())
params = {'KNN__n_neighbors': [1,2,3,4,5,6,7,8,9,10],
          'SCL':[std_scaler, mm_scaler, None]}

gcv = GridSearchCV(pipe, param_grid=params, cv=kfold, scoring='neg_log_loss')

gcv.fit(X, y)

print(gcv.best_params_)
print(gcv.best_score_)

{'memory': None, 'steps': [('SCL', None), ('KNN', KNeighborsClassifier())], 'verbose': False, 'SCL': None, 'KNN': KNeighborsClassifier(), 'KNN__algorithm': 'auto', 'KNN__leaf_size': 30, 'KNN__metric': 'minkowski', 'KNN__metric_params': None, 'KNN__n_jobs': None, 'KNN__n_neighbors': 5, 'KNN__p': 2, 'KNN__weights': 'uniform'}
{'KNN__n_neighbors': 8, 'SCL': None}
-2.1501310079676905


In [10]:
best_model = gcv.best_estimator_
# Unlabelled data
test_conc = pd.read_csv(r'..\Cases\Glass Identification\tst_Glass.csv') 
test_conc.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.5321,14.0,0.00,0.34,70.23,0.001,6.7,1.23,0.000
1,1.5212,15.0,3.00,1.23,75.90,0.100,7.0,0.00,0.440
2,1.5112,13.0,3.50,2.30,73.00,3.400,14.0,2.30,0.220
3,1.5000,12.4,1.23,3.22,74.22,4.500,10.0,3.10,0.100
4,1.5200,13.0,2.40,0.34,71.22,3.200,9.0,1.44,0.001


In [11]:
pred_strength = best_model.predict(test_conc)
print(pred_strength)

[3 1 2 3 0 3]


In [12]:
pred_type = le.inverse_transform(pred_strength)
print(pred_type)

['headlamps' 'building_windows_non_float_processed' 'containers'
 'headlamps' 'building_windows_float_processed' 'headlamps']
